In [ ]:
from datetime import datetime
# 打印当前时间，按照年-月-日 小时:分钟:秒的格式
print("Start time:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

# 记录开始时间
from time import time
start = time()

# 绘制校准曲线：语义熵与准确率

In [ ]:
%load_ext autoreload
%autoreload 2

import json
import warnings
import os
import pickle
from tqdm import tqdm
import torch
from collections import defaultdict, Counter
import argparse
from core.models.entailment import EntailmentDeberta
from core.data.data_utils import load_ds_from_json
import numpy as np
import pandas as pd
import scipy
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, roc_auc_score, mean_squared_error
from sklearn.model_selection import train_test_split

import csv
import jsonlines
import json
import torch
from torch import nn, optim
from transformers import AutoTokenizer, GPT2LMHeadModel, GPT2Tokenizer, GPTJForCausalLM
from tqdm import tqdm
import numpy as np
import os

from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
import pandas as pd
import scipy.stats

warnings.filterwarnings("ignore")
plt.rcParams.update({'font.size': 12})
rng = np.random.default_rng(42)


def load_pickle_file(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data

def save_pickle_file(file_path, data):
    with open(file_path, 'wb') as f:
        pickle.dump(data, f)

In [ ]:
def plot_calibration(probe_preds):
    p_true = probe_preds
    p_false = 1 - p_true
    calib_range = []
    calib_acc = []
    calib_size = []
    step = 0.1
    for start in np.arange(0, 1 + step, step):
        end = start + step
        pred_true = ((start <= p_true) & (p_true < end)).sum()
        pred_false = ((start <= p_false) & (p_false < end)).sum()
        if pred_true + pred_false == 0:
            continue
        frac_true = pred_true / (pred_true + pred_false)
        calib_range.append(start)
        calib_acc.append(frac_true)
        calib_size.append(pred_true + pred_false)

    data = pd.DataFrame({"range": calib_range, "acc": calib_acc, "size": calib_size})
    fig, ax = plt.subplots(figsize=(3, 3))
    sns.scatterplot(ax=ax, data=data, x="range", y="acc", size="size")
    ax.get_legend().remove()

    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.show()

# for dataset in data:
#     print(dataset)
#     plot_calibration(probe_preds[dataset])
#     plot_calibration(torch.sigmoid(data_processed[dataset]["valid"]["ratio"].float()).detach().cpu().numpy())